In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [3]:
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [4]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [5]:
input_size = 28
hidden_size = 128
output_size = 10
num_layers = 2
num_epochs = 10
learning_rate = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.squeeze(1)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


In [7]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.squeeze(1) 
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

In [8]:
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294173
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.593215
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.226385
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.302779
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.217628
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.122559
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.048591
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.035125
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.059223
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.145845

Test set: Average loss: 0.0001, Accuracy: 9678/10000 (96.78%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.076987
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.109313
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.032973
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.095168
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.012258
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.094581
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.053296
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.147064
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.03626